In [1]:
!pip install selenium
!apt-get update
!apt-get install firefox -y
!pip install selenium
# Download and install GeckoDriver
!wget https://github.com/mozilla/geckodriver/releases/download/v0.30.0/geckodriver-v0.30.0-linux64.tar.gz
!tar -xvzf geckodriver-v0.30.0-linux64.tar.gz
!chmod +x geckodriver
!mv geckodriver /usr/local/bin/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB

In [2]:
import pandas as pd
import gdown
import re
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from selenium import webdriver
import os
from google.colab import drive
import gc

In [3]:
gdown.download(id = '1DuT3nnIouLKuRFbGo5lqcQwKTTb5GM8r')
df = pd.read_csv('/content/suburbs.csv')

Downloading...
From: https://drive.google.com/uc?id=1DuT3nnIouLKuRFbGo5lqcQwKTTb5GM8r
To: /content/suburbs.csv
100%|██████████| 975k/975k [00:00<00:00, 93.1MB/s]


In [4]:
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Files-Required/client project')

Mounted at /content/drive


In [5]:
df.shape

(15879, 2)

In [6]:
df

,0,source
0,oaks-tas-7303,https://www.domain.com.au/suburb-profile/
1,gibraltar-range-nsw-2370,https://www.domain.com.au/suburb-profile/
2,bulli-creek-qld-4357,https://www.domain.com.au/suburb-profile/
3,gurindji-nt-0852,https://www.domain.com.au/suburb-profile/
4,buckenderra-nsw-2630,https://www.domain.com.au/suburb-profile/
...,...,...
15874,bindi-vic-3896,https://www.domain.com.au/suburb-profile/
15875,mongarlowe-nsw-2622,https://www.domain.com.au/suburb-profile/
15876,teal-point-vic-3579,https://www.domain.com.au/suburb-profile/
15877,macdonalds-creek-nsw-2339,https://www.domain.com.au/suburb-profile/


In [7]:
def driver_setup():
  options = webdriver.FirefoxOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Firefox(options=options)
  return driver

In [8]:
df_urbdat = pd.DataFrame()

In [9]:
for i in range(5290,6348):
  print(i)
  try:
    url = 'https://www.domain.com.au/suburb-profile/'+df['0'][i]
    driver = driver_setup()
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find("table")
    rows = table.find_all("tr")[1:]
    data = []

    for row in rows:
        cells = row.find_all("td")
        bedrooms = cells[0].text.strip()
        prop_type = cells[1].text.strip()
        median_price = cells[2].text.strip()
        days_on_market = cells[3].text.strip()
        clearance_rate = cells[4].text.strip()
        sold_this_year = cells[5].text.strip()
        population = soup.find('div', {'class': 'css-48zwbo'}).find('div', {'class': 'css-54bw0x'}).text
        average_age = soup.find_all('div', {'class': 'css-48zwbo'})[1].find('div', {'class': 'css-54bw0x'}).text
        owner_percent = soup.find('span', {'data-testid': 'left-value'}).text.strip()
        renter_percent = soup.find('span', {'data-testid': 'right-value'}).text.strip()
        family_percent = soup.find_all('span', {'data-testid': 'left-value'})[1].text.strip()
        single_percent = soup.find_all('span', {'data-testid': 'right-value'})[1].text.strip()

        row_data = {
            "Bedrooms": bedrooms,
            "Type": prop_type,
            "Median Price": median_price,
            "Avg Days on Market": days_on_market,
            "Clearance Rate": clearance_rate,
            "Sold This Year": sold_this_year,
            'POPULATION': population,
            'AVERAGE AGE': average_age,
            'OWNER': owner_percent,
            'RENTER': renter_percent,
            'FAMILY': family_percent,
            'SINGLE': single_percent,
            'SUBURB': df['0'][i]
        }

        data.append(row_data)

    tempdf = pd.DataFrame(data)
    df_urbdat = pd.concat([df_urbdat,tempdf],ignore_index =True)
  except:
    None
  driver.quit()
  gc.collect()
  os.system("taskkill /f /im firefox.exe")

5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368
5369
5370
5371
5372
5373
5374
5375
5376
5377
5378
5379
5380
5381
5382
5383
5384
5385
5386
5387
5388
5389
5390
5391
5392
5393
5394
5395
5396
5397
5398
5399
5400
5401
5402
5403
5404
5405
5406
5407
5408
5409
5410
5411
5412
5413
5414
5415
5416
5417
5418
5419
5420
5421
5422
5423
5424
5425
5426
5427
5428
5429
5430
5431
5432
5433
5434
5435
5436
5437
5438
5439
5440
5441
5442
5443
5444
5445
5446
5447
5448
5449
5450
5451
5452
5453
5454
5455
5456
5457
5458
5459
5460
5461
5462
5463
5464
5465
5466
5467
5468
5469
5470
5471
5472
5473
5474
5475
5476
5477
5478
5479
5480
5481
5482
5483
5484
5485
5486
5487
5488
5489


In [10]:
df_urbdat.to_csv('df_urbdat1.csv', index=False)

In [11]:
# def extract_first_name(name):
#     # Remove special characters using regex
#     cleaned_name = re.sub('[^a-zA-Z ]', '', name)
#     # Extract the first name
#     first_name = cleaned_name.split()[0]
#     return first_name

# def extract_first_names(names):
#     # Split the names by comma
#     name_list = names.split(',')
#     # Extract the first name from each name
#     first_names = [extract_first_name(name) for name in name_list]
#     return first_names

In [12]:
# print(len(unique_names))